In [1]:
import pandas as pd
import numpy as np
import itertools
from itertools import combinations

In [1632]:
df_loaded = pd.read_pickle(r"C:\Users\xhmiaud03\Desktop\Freight\All_in_one_Freight.pkl")

In [1050]:
Freight = df_loaded.copy()

In [1051]:
# Freight = Freight.groupby('Shipment n').filter(lambda grp: grp['City Code'].nunique() > 1)
Freight = Freight.groupby('Shipment n').filter(lambda grp: grp['City Code'].nunique() <= 1)

In [1052]:
Freight['Gate Out D'] = (pd.to_datetime
                                 (Freight['Gate Out D'], errors='coerce'))
Freight['Gate Out D'] = Freight['Gate Out D'].dt.date

In [1053]:
len(Freight)

650894

In [1054]:
Freight = Freight[Freight['Truck No'].astype(str).str.startswith(('U','D','F'))]
Freight.reset_index(drop=True, inplace=True)
Freight.index += 1

In [1055]:
len(Freight)

558769

In [1056]:
Freight = Freight.loc[Freight['City Code'] == 'E10']
Freight.reset_index(drop=True, inplace=True)
Freight.index += 1
len(Freight)

14818

In [1057]:
Freight.sort_values(by='Gate Out D', inplace=True)
Freight.reset_index(drop=True, inplace=True)
Freight.index += 1

In [1058]:
Freight['Gate Out D'] = (pd.to_datetime
                                 (Freight['Gate Out D'], errors='coerce'))
# Freight['year'] = Freight['Gate Out D'].dt.year
# Freight['month'] = Freight['Gate Out D'].dt.month
# Freight['day'] = Freight['Gate Out D'].dt.day

In [1059]:
Freight = Freight[(Freight['Gate Out D'].dt.month == 12) & (Freight['Gate Out D'].dt.year == 2023)]
Freight.reset_index(drop=True, inplace=True)
Freight.index += 1

In [1060]:
len(Freight)

392

In [1061]:
Freight['Gate Out D'] = Freight['Gate Out D'].dt.date
Freight.reset_index(drop=True, inplace=True)
Freight.index += 1

In [4]:
data = {
    'Model Name': ['Grand i10','i20','Aura/Xcent','Creta','Ioniq','Exter','Verna','Tuscon','Venue','Kona','New Verna','Santro','Elantra'],
    'Model Code': ['HQA','SVA','HQF','FHW','6IW','HQW','H6F','9ML','SPW','HOW','0YF','C4A','S5F'],
    'Length(m)':    [3.765,3.995,3.995,4.330,4.635,3.815,4.535,4.480,3.995,4.180,4.535,3.610,4.570],
    'Width(m)':     [1.660,1.775,1.680,1.790,1.890,1.710,1.765,1.850,1.770,1.800,1.765,1.645,1.800],
}


# Create DataFrame
df = pd.DataFrame(data)
df['Area(m)'] = df['Length(m)'] * df['Width(m)']
# df.loc[df['Model Name'].isin(['Aura', 'Xcent']), 'Model Code'] = 'SVA'
# df['Model Code'] = df['Model Code'].replace('ALL', 'SVA')
cardim = df.copy()

In [5]:
cardim

,Model Name,Model Code,Length(m),Width(m),Area(m)
0,Grand i10,HQA,3.765,1.660,6.249900
1,i20,SVA,3.995,1.775,7.091125
2,Aura/Xcent,HQF,3.995,1.680,6.711600
3,Creta,FHW,4.330,1.790,7.750700
4,Ioniq,6IW,4.635,1.890,8.760150
5,Exter,HQW,3.815,1.710,6.523650
6,Verna,H6F,4.535,1.765,8.004275
7,Tuscon,9ML,4.480,1.850,8.288000
8,Venue,SPW,3.995,1.770,7.071150
9,Kona,HOW,4.180,1.800,7.524000


In [1064]:
column_names_list = cardim.columns.tolist()
print(column_names_list)

['Model Name', 'Model Code', 'Length(m)', 'Width(m)', 'Area(m)']


In [1065]:
cardimcardim = cardim[['Model Name', 'Model Code', 'Length(m)', 'Width(m)', 'Area(m)']]
mapping = dict(zip(cardim['Model Code'], cardim['Model Name']))
Freight['Model Name'] = Freight['Model Code'].map(mapping)
mapping = dict(zip(cardim['Model Code'], cardim['Length(m)']))
Freight['Length(m)'] = Freight['Model Code'].map(mapping)
mapping = dict(zip(cardim['Model Code'], cardim['Width(m)']))
Freight['Width(m)'] = Freight['Model Code'].map(mapping)
mapping = dict(zip(cardim['Model Code'], cardim['Area(m)']))
Freight['Area(m)'] = Freight['Model Code'].map(mapping)
condition = Freight['Model Code'].isin(['FHB', 'FHC','FHY'])
Freight.loc[condition, ['Model Name', 'Length(m)', 'Width(m)', 'Area(m)']] = ['Alcazar',4.500, 1.790,8.1445]
condition = Freight['Model Code'].isin(['HRW'])
Freight.loc[condition, ['Model Name', 'Length(m)', 'Width(m)', 'Area(m)']] = ['Tuscon',4.480,1.850,8.288000]
condition = Freight['Model Code'].isin(['B4F'])
Freight.loc[condition, ['Model Name', 'Length(m)', 'Width(m)', 'Area(m)']] = ['Aura/Xcent',3.995,1.680,6.711600]

In [1067]:
Best_Shippment = Freight.groupby(['Shipment n', 'Truck No'])['Area(m)'].sum().reset_index()
Best_Shippment['Total_Area(m)'] = Best_Shippment['Area(m)']
Best_Shippment = Best_Shippment.drop_duplicates(subset=['Shipment n'], keep='first')
Best_Shippment = Best_Shippment.sort_values(by='Total_Area(m)', ascending=False)
Best_Shippment.drop(columns=['Area(m)'], inplace=True)
Best_Shippment.reset_index(drop=True, inplace=True)
Best_Shippment.index += 1
Area_ = Best_Shippment.copy()

In [1068]:
Best_Shippment.drop(columns=['Total_Area(m)'], inplace=True)

In [1069]:
column_list = ['Grand i10','i20','Aura/Xcent','Creta','Ioniq','Exter','Verna','Tuscon','Venue','Kona','New Verna','Santro','Elantra','Alcazar']
for col in column_list:
    Best_Shippment[col] = None
# Initialize the columns in df2 with zeros
for col in Best_Shippment.columns:
    if col not in ['Shipment n', 'Truck No']:
        Best_Shippment[col] = 0

# Update df2 based on df1
for index, row in Freight.iterrows():
    mask = (Best_Shippment['Shipment n'] == row['Shipment n']) & (Best_Shippment['Truck No'] == row['Truck No'])
    Best_Shippment.loc[mask, row['Model Name']] += 1

In [1070]:
Best_of_Freight = Best_Shippment.copy()

In [1071]:
Best_Shippment = Best_Shippment.merge(Freight[['Gate Out D', 'Shipment n']], on='Shipment n', how='left')
Best_Shippment = Best_Shippment.merge(Area_[['Total_Area(m)', 'Shipment n']], on='Shipment n', how='left')
Best_Shippment = Best_Shippment.drop_duplicates(subset=['Shipment n'], keep='first')
Best_Shippment['total_cars'] = Best_Shippment[['Grand i10','i20','Aura/Xcent','Creta','Ioniq','Exter','Verna','Tuscon','Venue','Kona','New Verna','Santro','Elantra','Alcazar']].sum(axis=1)
Best_Shippment = Best_Shippment[['Gate Out D', 'Shipment n', 'Truck No','total_cars','Total_Area(m)','Grand i10', 'i20', 'Aura/Xcent', 'Creta', 'Ioniq', 'Exter', 'Verna', 'Tuscon', 'Venue', 'Kona', 'New Verna', 'Santro', 'Elantra', 'Alcazar']]
Best_Shippment.reset_index(drop=True, inplace=True)
Best_Shippment.index += 1

In [1072]:
df2 = Best_Shippment.copy()

In [1073]:
# Assuming you have already loaded df1 and df2

# Create a dictionary to map model names to their lengths and widths
model_info = {
   'Grand i10': {'Length(m)': 3.765, 'Width(m)': 1.660},
    'i20': {'Length(m)': 3.995, 'Width(m)': 1.775},
    'Aura/Xcent': {'Length(m)': 3.995, 'Width(m)': 1.680},
    'Creta': {'Length(m)': 4.330, 'Width(m)': 1.790},
    'Ioniq': {'Length(m)': 4.635, 'Width(m)': 1.890},
    'Exter': {'Length(m)': 3.815, 'Width(m)': 1.710},
    'Verna': {'Length(m)': 4.535, 'Width(m)': 1.765},
    'Tuscon': {'Length(m)': 4.480, 'Width(m)': 1.850},
    'Venue': {'Length(m)': 3.995, 'Width(m)': 1.770},
    'Kona': {'Length(m)': 4.180, 'Width(m)': 1.800},
    'New Verna': {'Length(m)': 4.535, 'Width(m)': 1.765},
    'Santro': {'Length(m)': 3.610, 'Width(m)': 1.645},
    'Elantra': {'Length(m)': 4.570, 'Width(m)': 1.800},
    'Alcazar': {'Length(m)': 4.500, 'Width(m)': 1.790}
}

# Calculate total length and total width for each row in df2
df2['total_length'] = df2.apply(lambda row: sum(model_info[model]['Length(m)'] * row[model] for model in model_info), axis=1)
df2['total_width'] = df2.apply(lambda row: sum(model_info[model]['Width(m)'] * row[model] for model in model_info), axis=1)

# Print the modified df2
df2
Best_Shippment = df2


In [1076]:
Best_Shippment.to_pickle('Best_Shipment.pkl')
Best_Shippment.to_excel("Best_Shipment.xlsx", index=False)

In [1077]:
data = Best_Shippment.copy()

In [1078]:
column_names_list = data.columns.tolist()
print(column_names_list)

['Gate Out D', 'Shipment n', 'Truck No', 'total_cars', 'Total_Area(m)', 'Grand i10', 'i20', 'Aura/Xcent', 'Creta', 'Ioniq', 'Exter', 'Verna', 'Tuscon', 'Venue', 'Kona', 'New Verna', 'Santro', 'Elanta', 'Alcazar', 'total_length', 'total_width']


In [1079]:
column_names = ['Grand i10','i20','Aura/Xcent','Creta','Ioniq','Exter','Verna','Tuscon','Venue','Kona','New Verna','Santro','Elantra','Alcazar','total_cars']

# Initialize an empty dictionary to store the sums
column_sums = {}

# Iterate through each column name
for col_name in column_names:
    column_sums[col_name] = Best_Shippment[col_name].sum()

# Print the sums
for col_name, col_sum in column_sums.items():
    print(f"{col_name}: {col_sum}")

Grand i10: 110
i20: 59
Aura/Xcent: 16
Creta: 55
Ioniq: 0
Exter: 66
Verna: 0
Tuscon: 0
Venue: 56
Kona: 0
New Verna: 24
Santro: 0
Elanta: 0
Alcazar: 6
total_cars: 392


In [1441]:
unique_count = data['Shipment n'].nunique()
print(f"Number of unique values in column 'Shipment n': {unique_count}")

Number of unique values in column 'Shipment n': 50


In [1442]:
# # Merge the two DataFrames on 'Model Name'
# df = pd.merge(df1, df2, on='Model Name')

# # Sort the DataFrame by area in descending order
# df = df.sort_values(by='Area(m)', ascending=False)

# # Initialize variables
# trucks = []
# current_truck = []
# current_area = 0
# current_count = 0

# # Function to add a car to the current truck
# def add_to_truck(row):
#     global current_truck, current_area, current_count
#     current_truck.append(row['Model Name'])
#     current_area += row['Area(m)']
#     current_count += 1

# # Iterate through the DataFrame
# for index, row in df.iterrows():
#     for _ in range(row['Count']):
#         if current_count < 8 and current_area + row['Area(m)'] <= 60.22:
#             add_to_truck(row)
#         else:
#             trucks.append(current_truck)
#             current_truck = []
#             current_area = 0
#             current_count = 0
#             add_to_truck(row)

# # Add the last truck if it has cars
# if current_truck:
#     trucks.append(current_truck)

# # Print the result
# for i, truck in enumerate(trucks):
#     print(f"Truck {i+1}: {truck}")

In [109]:
df1 = cardim[['Model Name','Length(m)', 'Width(m)','Area(m)']]

In [110]:
df1 = df1[['Model Name','Length(m)', 'Width(m)','Area(m)']]
new_row_df = {'Model Name': 'Alcazar','Length(m)':4.500, 'Width(m)':1.790,'Area(m)': 8.1445}
# Append the new row to the DataFrame
new_row_df = pd.DataFrame([new_row_df])
# Concatenate the new row to the existing DataFrame
df1 = pd.concat([df1, new_row_df], ignore_index=True)

In [111]:
df1

,Model Name,Length(m),Width(m),Area(m)
0,Grand i10,3.765,1.660,6.249900
1,i20,3.995,1.775,7.091125
2,Aura/Xcent,3.995,1.680,6.711600
3,Creta,4.330,1.790,7.750700
4,Ioniq,4.635,1.890,8.760150
5,Exter,3.815,1.710,6.523650
6,Verna,4.535,1.765,8.004275
7,Tuscon,4.480,1.850,8.288000
8,Venue,3.995,1.770,7.071150
9,Kona,4.180,1.800,7.524000


In [112]:
df1 = df1[['Model Name','Length(m)', 'Width(m)','Area(m)']]
new_row_df = {'Model Name': 'Alcazar','Length(m)':4.500, 'Width(m)':1.790, 'Area(m)': 8.1445}
# Append the new row to the DataFrame
new_row_df = pd.DataFrame([new_row_df])

# Concatenate the new row to the existing DataFrame
df1 = pd.concat([df1, new_row_df], ignore_index=True)

In [113]:
# data2 = {
#     'Model Name': [
#     "Grand i10", "i20", "Aura/Xcent", "Creta", "Ioniq",
#     "Exter", "Verna", "Tuscon", "Venue", "Kona",
#     "New Verna", "Santro", "Elantra", "Alcazar"
# ],
#     'Count':[
#     110, 59, 16, 55, 0,
#     66, 0, 0, 56, 0,
#     24, 0, 0, 6
# ]
# }
# df2 = pd.DataFrame(data2)

In [114]:
data2 = {
    'Model Name': [
    "Grand i10", "i20", "Aura/Xcent", "Creta", "Ioniq",
    "Exter", "Verna", "Tuscon", "Venue", "Kona",
    "New Verna", "Santro", "Elantra", "Alcazar"
],
    'Count':[
    1, 2, 0, 5, 7,
    0, 0, 0, 0, 0,
    0, 0, 0, 6
]
}
df2 = pd.DataFrame(data2)

In [115]:
df = pd.merge(df1, df2, on='Model Name')

# # Sort the DataFrame by area in descending order
# df = df.sort_values(by='Area(m)', ascending=False)

# # Initialize variables
# trucks = []
# current_truck = {model: 0 for model in df['Model Name']}
# current_area = 0
# current_count = 0
# truck_no = 1

# # Function to add a car to the current truck
# def add_to_truck(row):
#     global current_truck, current_area, current_count
#     current_truck[row['Model Name']] += 1
#     current_area += row['Area(m)']
#     current_count += 1

# # Iterate through the DataFrame
# for index, row in df.iterrows():
#     for _ in range(row['Count']):
#         if current_count < 8 and current_area + row['Area(m)'] <= 58.12:
#             add_to_truck(row)
#         else:
#             current_truck['Truck No'] = truck_no
#             trucks.append(current_truck)
#             truck_no += 1
#             current_truck = {model: 0 for model in df['Model Name']}
#             current_area = 0
#             current_count = 0
#             add_to_truck(row)

# # Add the last truck if it has cars
# if current_count > 0:
#     current_truck['Truck No'] = truck_no
#     trucks.append(current_truck)

# # Create the final DataFrame
# truck_df = pd.DataFrame(trucks)

# # # Display the DataFrame
# # truck_df

In [116]:
# df2 = cardim.copy()

In [118]:
data2 = {
    'Model Name': [
    "Grand i10", "i20", "Aura/Xcent", "Creta", "Ioniq",
    "Exter", "Verna", "Tuscon", "Venue", "Kona",
    "New Verna", "Santro", "Elantra", "Alcazar"
],
    'Count':[
    1, 2, 0, 5, 7,
    0, 0, 0, 0, 0,
    0, 0, 0, 6
]
}
df2 = pd.DataFrame(data2)

In [101]:
# truck_df2 = truck_df.copy()
# # Append the new row to the DataFrame
# df2 = df2[['Model Name','Length(m)', 'Width(m)','Area(m)']]
# new_row_df = {'Model Name': 'Alcazar','Length(m)':4.500, 'Width(m)':1.790, 'Area(m)': 8.1445}
# # Append the new row to the DataFrame
# new_row_df = pd.DataFrame([new_row_df])

# # Concatenate the new row to the existing DataFrame
# df2 = pd.concat([df2, new_row_df], ignore_index=True)

In [102]:
# Append the new row to the DataFrame
df1 = df1[['Model Name','Length(m)', 'Width(m)','Area(m)']]
new_row_df = {'Model Name': 'Alcazar','Length(m)':4.500, 'Width(m)':1.790, 'Area(m)': 8.1445}
# Append the new row to the DataFrame
new_row_df = pd.DataFrame([new_row_df])

# Concatenate the new row to the existing DataFrame
df1 = pd.concat([df1, new_row_df], ignore_index=True)

In [119]:
# import pandas as pd
# data2 = {
#     'Model Name': [
#     "Grand i10", "i20", "Aura/Xcent", "Creta", "Ioniq",
#     "Exter", "Verna", "Tuscon", "Venue", "Kona",
#     "New Verna", "Santro", "Elantra", "Alcazar"
# ],
#     'Count':[
#     110, 59, 16, 55, 0,
#     66, 0, 0, 56, 0,
#     24, 0, 0, 6
# ]
# }
# df2 = pd.DataFrame(data2)
# Assuming df1 and df2 are already defined
# Append the new row to the DataFrame
df1 = df1[['Model Name','Length(m)', 'Width(m)','Area(m)']]
new_row_df = {'Model Name': 'Alcazar','Length(m)':4.500, 'Width(m)':1.790, 'Area(m)': 8.1445}
# Append the new row to the DataFrame
new_row_df = pd.DataFrame([new_row_df])

# Concatenate the new row to the existing DataFrame
df1 = pd.concat([df1, new_row_df], ignore_index=True)
# Merge the DataFrames on 'Model Name'
df = pd.merge(df1, df2, on='Model Name')

# Sort the DataFrame by area in descending order
df = df.sort_values(by='Area(m)', ascending=False)
df = df.head()

In [107]:
trucks = []
current_truck = {model: 0 for model in df['Model Name']}
current_length = 0
current_width = 0
current_area = 0
current_count = 0
truck_no = 1

# Function to add a car to the current truck
def add_to_truck(row):
    global current_truck, current_area, current_count, current_length, current_width
    current_truck[row['Model Name']] += 1
    current_area += row['Area(m)']
    current_width += row['Width(m)']
    current_length += row['Length(m)']
    current_count += 1

# Iterate through the DataFrame
for index, row in df.iterrows():
    for _ in range(row['Count']):
        if (current_count <= 8 and
            current_area + row['Area(m)'] <= 60.22 and
            current_length + row['Length(m)'] <= 34.5):
            add_to_truck(row)
        else:
            current_truck['Truck No'] = truck_no
            trucks.append(current_truck)
            truck_no += 1
            current_truck = {model: 0 for model in df['Model Name']}
            current_length = 0
            current_width = 0
            current_area = 0
            current_count = 0
            add_to_truck(row)

# Add the last truck if it has cars
if current_count > 0:
    current_truck['Truck No'] = truck_no
    trucks.append(current_truck)

# Create the final DataFrame
truck_df = pd.DataFrame(trucks)

# Display the DataFrame
print(truck_df)

   Ioniq  Tuscon  Elanta  Alcazar  Truck No
0      0       0       0        7         1
1      0       0       0        5         2


In [105]:
# Sort the DataFrame by area in descending order
df = df.sort_values(by='Area(m)', ascending=False)

# Initialize variables
trucks = []
current_truck = {model: 0 for model in df['Model Name']}
current_length, current_width, current_area, current_count = 0, 0, 0, 0
truck_no = 1

# Function to add a car to the current truck
def add_to_truck(row):
    global current_truck, current_area, current_count, current_length, current_width
    current_truck[row['Model Name']] += 1
    current_area += row['Area(m)']
    current_width += row['Width(m)']
    current_length += row['Length(m)']
    current_count += 1

# Iterate through the DataFrame
for _, row in df.iterrows():
    for _ in range(row['Count']):
        if (current_count <= 8 and
            current_area + row['Area(m)'] <= 60.22 and
            current_length + row['Length(m)'] <= 34.5 and
            current_width + row['Width(m)'] <= 14.2):
            add_to_truck(row)
        else:
            current_truck['Truck No'] = truck_no
            trucks.append(current_truck)
            truck_no += 1
            current_truck = {model: 0 for model in df['Model Name']}
            current_length, current_width, current_area, current_count = 0, 0, 0, 0
            add_to_truck(row)

# Add the last truck if it has cars
if current_count > 0:
    current_truck['Truck No'] = truck_no
    trucks.append(current_truck)

# Create the final DataFrame
truck_df = pd.DataFrame(trucks)

# Display the DataFrame
truck_df

,Ioniq,Tuscon,Elanta,Alcazar,Truck No
0,0,0,0,7,1
1,0,0,0,5,2


In [70]:
df = df.sort_values(by='Area(m)', ascending=False)

# Initialize variables
trucks = []
current_truck = {model: 0 for model in df['Model Name']}
current_length, current_width, current_area, current_count = 0, 0, 0, 0
truck_no = 1

# Function to add a car to the current truck
def add_to_truck(row):
    global current_truck, current_area, current_count, current_length, current_width
    current_truck[row['Model Name']] += 1
    current_area += row['Area(m)']
    current_width += row['Width(m)']
    current_length += row['Length(m)']
    current_count += 1

# Iterate through the DataFrame
for _, row in df.iterrows():
    for _ in range(row['Count']):
        if (current_count <= 8 and
            current_area + row['Area(m)'] <= 60.22 and
            current_length + row['Length(m)'] <= 34.5 and
            current_width + row['Width(m)'] <= 14.2):
            add_to_truck(row)
        else:
            current_truck['Truck No'] = truck_no
            trucks.append(current_truck)
            truck_no += 1
            current_truck = {model: 0 for model in df['Model Name']}
            current_length, current_width, current_area, current_count = 0, 0, 0, 0
            add_to_truck(row)

# Add the last truck if it has cars
if current_count > 0:
    current_truck['Truck No'] = truck_no
    trucks.append(current_truck)

# Create the final DataFrame
truck_df = pd.DataFrame(trucks)

# Display the DataFrame
truck_df

,Ioniq,Tuscon,Elanta,Alcazar,Truck No
0,0,0,0,7,1
1,0,0,0,5,2


In [59]:
# Sort the DataFrame by area in descending order
df = df.sort_values(by='Area(m)', ascending=False)

# Initialize variables
best_trucks = None
min_trucks = float('inf')

# Generate all combinations of cars
for r in range(1, len(df) + 1):
    for combo in combinations(df.iterrows(), r):
        current_truck = {model: 0 for model in df['Model Name']}
        current_length = 0
        current_width = 0
        current_area = 0
        current_count = 0

        for _, row in combo:
            for _ in range(row['Count']):
                if (current_count <= 8 and
                    current_area + row['Area(m)'] <= 60.220000 and
                    current_length + row['Length(m)'] <= 34.500000 and
                    current_width + row['Width(m)'] <= 14.2000):
                    add_to_truck(row)
                else:
                    break

        if current_count > 0:
            trucks_needed = len([val for val in current_truck.values() if val > 0])
            if trucks_needed < min_trucks:
                min_trucks = trucks_needed
                best_trucks = current_truck

# Create the final DataFrame
truck_df = pd.DataFrame([best_trucks])

# Display the DataFrame
truck_df

,Ioniq,Tuscon,Elanta,Alcazar
0,0,0,0,6


In [120]:
df = pd.merge(df1, df2, on='Model Name')

# Sort the DataFrame by area in descending order
df = df.sort_values(by='Area(m)', ascending=False)

# Initialize variables
trucks = []
current_truck = {model: 0 for model in df['Model Name']}
current_area = 0
current_count = 0
truck_no = 1

# Function to add a car to the current truck
def add_to_truck(row):
    global current_truck, current_area, current_count
    current_truck[row['Model Name']] += 1
    current_area += row['Area(m)']
    current_count += 1

# Iterate through the DataFrame
for index, row in df.iterrows():
    for _ in range(row['Count']):
        if current_area + row['Area(m)'] <= 60.00   and current_count <= 8 :
            add_to_truck(row)
        else:
            current_truck['Truck No'] = truck_no
            trucks.append(current_truck)
            truck_no += 1
            current_truck = {model: 0 for model in df['Model Name']}
            current_area = 0
            current_count = 0
            add_to_truck(row)

# Add the last truck if it has cars
if current_count > 0:
    current_truck['Truck No'] = truck_no
    trucks.append(current_truck)

# Create the final DataFrame
truck_df = pd.DataFrame(trucks)

# Display the DataFrame
truck_df

,Ioniq,Tuscon,Alcazar,Verna,New Verna,Creta,Kona,i20,Venue,Aura/Xcent,Exter,Grand i10,Santro,Truck No
0,6,0,0,0,0,0,0,0,0,0,0,0,0,1
1,1,0,6,0,0,0,0,0,0,0,0,0,0,2
2,0,0,7,0,0,0,0,0,0,0,0,0,0,3
3,0,0,5,0,0,2,0,0,0,0,0,0,0,4
4,0,0,0,0,0,3,0,2,0,0,0,1,0,5


In [10]:
column_names_list = truck_df.columns.tolist()
print(column_names_list)

['Ioniq', 'Tuscon', 'Elanta', 'Alcazar', 'Verna', 'New Verna', 'Creta', 'Kona', 'i20', 'Venue', 'Aura/Xcent', 'Exter', 'Grand i10', 'Santro', 'Truck No']


In [11]:
truck_df2 = truck_df.copy()

In [12]:
truck_df = truck_df[['Truck No','Ioniq', 'Tuscon', 'Elantra', 'Verna', 'New Verna', 'Creta', 'Kona', 'i20', 'Venue', 'Aura/Xcent', 'Grand i10', 'Santro', 'Exter','Alcazar']]
truck_df.reset_index(drop=True, inplace=True)
truck_df.index += 1

In [13]:
truck_df

,Truck No,Ioniq,Tuscon,Elanta,Verna,New Verna,Creta,Kona,i20,Venue,Aura/Xcent,Grand i10,Santro,Exter,Alcazar
1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,6
2,2,0,0,0,0,7,0,0,0,0,0,0,0,0,0
3,3,0,0,0,0,7,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,7,0,0,0,0,0,0,0,0,0
5,5,0,0,0,0,2,5,0,0,0,0,0,0,0,0
6,6,0,0,0,0,0,7,0,0,0,0,0,0,0,0
7,7,0,0,0,0,0,7,0,0,0,0,0,0,0,0
8,8,0,0,0,0,0,7,0,0,0,0,0,0,0,0
9,9,0,0,0,0,0,7,0,0,0,0,0,0,0,0
10,10,0,0,0,0,0,7,0,0,0,0,0,0,0,0


In [14]:
df2 = cardim.copy()

In [15]:
df2 = df2[['Model Name','Length(m)', 'Width(m)','Area(m)']]
new_row_df = {'Model Name': 'Alcazar','Length(m)':4.500, 'Width(m)':1.790,'Area(m)': 8.1445}
# Append the new row to the DataFrame
new_row_df = pd.DataFrame([new_row_df])
# Concatenate the new row to the existing DataFrame
df2 = pd.concat([df2, new_row_df], ignore_index=True)

In [16]:
# df2 = df2[['Model Name','Area(m)']]
# new_row_df = {'Model Name': 'Alcazar', 'Area(m)': 8.1445}
# # Append the new row to the DataFrame
# new_row_df = pd.DataFrame([new_row_df])

# # Concatenate the new row to the existing DataFrame
# df2 = pd.concat([df2, new_row_df], ignore_index=True)

In [17]:
df2

,Model Name,Length(m),Width(m),Area(m)
0,Grand i10,3.765,1.660,6.249900
1,i20,3.995,1.775,7.091125
2,Aura/Xcent,3.995,1.680,6.711600
3,Creta,4.330,1.790,7.750700
4,Ioniq,4.635,1.890,8.760150
5,Exter,3.815,1.710,6.523650
6,Verna,4.535,1.765,8.004275
7,Tuscon,4.480,1.850,8.288000
8,Venue,3.995,1.770,7.071150
9,Kona,4.180,1.800,7.524000


In [18]:
model_length = df2.set_index('Model Name')['Length(m)'].to_dict()
model_width = df2.set_index('Model Name')['Width(m)'].to_dict()

# Calculate total length and total width for each truck
truck_df['totallength'] = truck_df.apply(lambda row: sum(row[model] * model_length[model] for model in model_length if model in row), axis=1)
truck_df['totalwidth'] = truck_df.apply(lambda row: sum(row[model] * model_width[model] for model in model_width if model in row), axis=1)

# Display the updated DataFrame
truck_df

C:\Users\xhmiaud03\AppData\Local\Temp\ipykernel_141456\3437668960.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  truck_df['totallength'] = truck_df.apply(lambda row: sum(row[model] * model_length[model] for model in model_length if model in row), axis=1)
C:\Users\xhmiaud03\AppData\Local\Temp\ipykernel_141456\3437668960.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  truck_df['totalwidth'] = truck_df.apply(lambda row: sum(row[model] * model_width[model] for model in model_width if model in row), axi

,Truck No,Ioniq,Tuscon,Elanta,Verna,New Verna,Creta,Kona,i20,Venue,Aura/Xcent,Grand i10,Santro,Exter,Alcazar,totallength,totalwidth
1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,6,31.535,12.505
2,2,0,0,0,0,7,0,0,0,0,0,0,0,0,0,31.745,12.355
3,3,0,0,0,0,7,0,0,0,0,0,0,0,0,0,31.745,12.355
4,4,0,0,0,0,7,0,0,0,0,0,0,0,0,0,31.745,12.355
5,5,0,0,0,0,2,5,0,0,0,0,0,0,0,0,30.720,12.480
6,6,0,0,0,0,0,7,0,0,0,0,0,0,0,0,30.310,12.530
7,7,0,0,0,0,0,7,0,0,0,0,0,0,0,0,30.310,12.530
8,8,0,0,0,0,0,7,0,0,0,0,0,0,0,0,30.310,12.530
9,9,0,0,0,0,0,7,0,0,0,0,0,0,0,0,30.310,12.530
10,10,0,0,0,0,0,7,0,0,0,0,0,0,0,0,30.310,12.530


In [19]:
# Create a dictionary for model areas
model_area = df2.set_index('Model Name')['Area(m)'].to_dict()

# Calculate total area for each truck
truck_df['totalarea'] = truck_df.apply(lambda row: sum(row[model] * model_area[model] for model in model_area if model in row), axis=1)

C:\Users\xhmiaud03\AppData\Local\Temp\ipykernel_141456\1713839763.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  truck_df['totalarea'] = truck_df.apply(lambda row: sum(row[model] * model_area[model] for model in model_area if model in row), axis=1)


In [20]:
truck_df['total_cars'] = truck_df[['Grand i10','i20','Aura/Xcent','Creta','Ioniq','Exter','Verna','Tuscon','Venue','Kona','New Verna','Santro','Elantra','Alcazar']].sum(axis=1)

C:\Users\xhmiaud03\AppData\Local\Temp\ipykernel_141456\2464101659.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  truck_df['total_cars'] = truck_df[['Grand i10','i20','Aura/Xcent','Creta','Ioniq','Exter','Verna','Tuscon','Venue','Kona','New Verna','Santro','Elanta','Alcazar']].sum(axis=1)


In [21]:
truck_df

,Truck No,Ioniq,Tuscon,Elanta,Verna,New Verna,Creta,Kona,i20,Venue,Aura/Xcent,Grand i10,Santro,Exter,Alcazar,totallength,totalwidth,totalarea,total_cars
1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,6,31.535,12.505,56.871275,7
2,2,0,0,0,0,7,0,0,0,0,0,0,0,0,0,31.745,12.355,56.029925,7
3,3,0,0,0,0,7,0,0,0,0,0,0,0,0,0,31.745,12.355,56.029925,7
4,4,0,0,0,0,7,0,0,0,0,0,0,0,0,0,31.745,12.355,56.029925,7
5,5,0,0,0,0,2,5,0,0,0,0,0,0,0,0,30.720,12.480,54.762050,7
6,6,0,0,0,0,0,7,0,0,0,0,0,0,0,0,30.310,12.530,54.254900,7
7,7,0,0,0,0,0,7,0,0,0,0,0,0,0,0,30.310,12.530,54.254900,7
8,8,0,0,0,0,0,7,0,0,0,0,0,0,0,0,30.310,12.530,54.254900,7
9,9,0,0,0,0,0,7,0,0,0,0,0,0,0,0,30.310,12.530,54.254900,7
10,10,0,0,0,0,0,7,0,0,0,0,0,0,0,0,30.310,12.530,54.254900,7


In [22]:
truck_df2 = truck_df.copy()

In [23]:
column_names = ['Grand i10','i20','Aura/Xcent','Creta','Ioniq','Exter','Verna','Tuscon','Venue','Kona','New Verna','Santro','Elantra','Alcazar','total_cars']

# Initialize an empty dictionary to store the sums
column_sums = {}

# Iterate through each column name
for col_name in column_names:
    column_sums[col_name] = truck_df[col_name].sum()

# Print the sums
for col_name, col_sum in column_sums.items():
    print(f"{col_name}: {col_sum}")

Grand i10: 110
i20: 59
Aura/Xcent: 16
Creta: 55
Ioniq: 0
Exter: 66
Verna: 0
Tuscon: 0
Venue: 56
Kona: 0
New Verna: 24
Santro: 0
Elanta: 0
Alcazar: 6
total_cars: 392


In [24]:
column_names_list = truck_df.columns.tolist()
print(column_names_list)

['Truck No', 'Ioniq', 'Tuscon', 'Elanta', 'Verna', 'New Verna', 'Creta', 'Kona', 'i20', 'Venue', 'Aura/Xcent', 'Grand i10', 'Santro', 'Exter', 'Alcazar', 'totallength', 'totalwidth', 'totalarea', 'total_cars']


In [25]:
truck_df = truck_df[['Truck No','totalarea','totallength', 'totalwidth','total_cars','Ioniq', 'Tuscon', 'Elantra', 'Verna', 'New Verna', 'Creta', 'Kona', 'i20', 'Venue', 'Aura/Xcent', 'Grand i10', 'Santro', 'Exter','Alcazar']]

In [26]:
truck_df

,Truck No,totalarea,totallength,totalwidth,total_cars,Ioniq,Tuscon,Elanta,Verna,New Verna,Creta,Kona,i20,Venue,Aura/Xcent,Grand i10,Santro,Exter,Alcazar
1,1,56.871275,31.535,12.505,7,0,0,0,0,1,0,0,0,0,0,0,0,0,6
2,2,56.029925,31.745,12.355,7,0,0,0,0,7,0,0,0,0,0,0,0,0,0
3,3,56.029925,31.745,12.355,7,0,0,0,0,7,0,0,0,0,0,0,0,0,0
4,4,56.029925,31.745,12.355,7,0,0,0,0,7,0,0,0,0,0,0,0,0,0
5,5,54.762050,30.720,12.480,7,0,0,0,0,2,5,0,0,0,0,0,0,0,0
6,6,54.254900,30.310,12.530,7,0,0,0,0,0,7,0,0,0,0,0,0,0,0
7,7,54.254900,30.310,12.530,7,0,0,0,0,0,7,0,0,0,0,0,0,0,0
8,8,54.254900,30.310,12.530,7,0,0,0,0,0,7,0,0,0,0,0,0,0,0
9,9,54.254900,30.310,12.530,7,0,0,0,0,0,7,0,0,0,0,0,0,0,0
10,10,54.254900,30.310,12.530,7,0,0,0,0,0,7,0,0,0,0,0,0,0,0


In [1613]:
Dimensions = cardim.copy()

In [1614]:
Dimensions = Dimensions[['Model Name','Length(m)', 'Width(m)']]
new_row_df = {'Model Name': 'Alcazar','Length(m)':4.500, 'Width(m)':1.790}
# Append the new row to the DataFrame
new_row_df = pd.DataFrame([new_row_df])
# Concatenate the new row to the existing DataFrame
Dimensions = pd.concat([Dimensions, new_row_df], ignore_index=True)

In [1615]:
Dimensions

,Model Name,Length(m),Width(m)
0,Grand i10,3.765,1.660
1,i20,3.995,1.775
2,Aura/Xcent,3.995,1.680
3,Creta,4.330,1.790
4,Ioniq,4.635,1.890
5,Exter,3.815,1.710
6,Verna,4.535,1.765
7,Tuscon,4.480,1.850
8,Venue,3.995,1.770
9,Kona,4.180,1.800


In [1616]:
Dimensions = Dimensions[['Model Name','Length(m)', 'Width(m)']]

In [1617]:
# truck_df2 = truck_df2[['Truck No', 'totalarea','totallength', 'totalwidth','total_cars','Ioniq', 'Tuscon', 'Elantra', 'Verna', 'New Verna', 'Creta', 'Kona', 'i20', 'Venue', 'Aura/Xcent', 'Grand i10', 'Santro', 'Exter', 'Alcazar']]

In [1618]:
truck_df2.head()

,Truck No,Ioniq,Tuscon,Elanta,Verna,New Verna,Creta,Kona,i20,Venue,Aura/Xcent,Grand i10,Santro,Exter,Alcazar,totallength,totalwidth,totalarea,total_cars
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,7,31.5,12.53,57.0115,7
2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,7,31.5,12.53,57.0115,7
3,3,0,0,0,0,0,0,0,0,0,0,0,0,0,7,31.5,12.53,57.0115,7
4,4,0,0,0,0,0,0,0,0,0,0,0,0,0,7,31.5,12.53,57.0115,7
5,5,0,0,0,0,0,0,0,0,0,0,0,0,0,7,31.5,12.53,57.0115,7


In [1619]:
truck_df2 = truck_df2.sort_values(by='totalarea', ascending=False)

In [1620]:
truck_df2.tail()

,Truck No,Ioniq,Tuscon,Elanta,Verna,New Verna,Creta,Kona,i20,Venue,Aura/Xcent,Grand i10,Santro,Exter,Alcazar,totallength,totalwidth,totalarea,total_cars
20,20,0,0,0,0,0,0,0,7,0,0,0,0,0,0,27.965,12.425,49.637875,7
21,21,0,0,0,0,0,0,0,7,0,0,0,0,0,0,27.965,12.425,49.637875,7
22,22,0,0,0,0,0,0,0,7,0,0,0,0,0,0,27.965,12.425,49.637875,7
23,23,0,0,0,0,0,0,0,7,0,0,0,0,0,0,27.965,12.425,49.637875,7
57,57,0,0,0,0,0,0,0,0,0,0,5,0,0,0,18.825,8.300,31.249500,5


In [1621]:
truck_df2.to_pickle('Optimized_Shippment.pkl')
truck_df2.to_excel("Optimized_Shippment.xlsx", index=False)

In [1622]:
truck_df2

,Truck No,Ioniq,Tuscon,Elanta,Verna,New Verna,Creta,Kona,i20,Venue,Aura/Xcent,Grand i10,Santro,Exter,Alcazar,totallength,totalwidth,totalarea,total_cars
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,7,31.500,12.530,57.011500,7
3,3,0,0,0,0,0,0,0,0,0,0,0,0,0,7,31.500,12.530,57.011500,7
4,4,0,0,0,0,0,0,0,0,0,0,0,0,0,7,31.500,12.530,57.011500,7
5,5,0,0,0,0,0,0,0,0,0,0,0,0,0,7,31.500,12.530,57.011500,7
6,6,0,0,0,0,0,0,0,0,0,0,0,0,0,7,31.500,12.530,57.011500,7
2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,7,31.500,12.530,57.011500,7
26,26,0,0,0,0,0,0,0,5,3,0,0,0,0,0,31.960,14.185,56.669075,8
32,32,0,0,0,0,0,0,0,0,8,0,0,0,0,0,31.960,14.160,56.569200,8
31,31,0,0,0,0,0,0,0,0,8,0,0,0,0,0,31.960,14.160,56.569200,8
30,30,0,0,0,0,0,0,0,0,8,0,0,0,0,0,31.960,14.160,56.569200,8


In [1623]:
df2 = truck_df2.copy()

In [1624]:
# # Assuming you have already loaded your dataframes df1 and df2

# # Create a dictionary with vehicle model names as keys and their lengths and widths as values
# vehicle_lengths = {
#     'Ioniq': 4.635,
#     'Tuscon': 4.480,
#     'Elantra': 4.570,
#     'Verna': 4.535,
#     'New Verna': 4.535,
#     'Creta': 4.330,
#     'Kona': 4.180,
#     'i20': 3.995,
#     'Venue': 3.995,
#     'Aura/Xcent': 3.995,
#     'Grand i10': 3.765,
#     'Santro': 3.610,
#     'Exter': 3.815,
#     'Alcazar': 4.500
# }


# # Assuming you have a similar dictionary for vehicle widths
# vehicle_widths = {
#     'Grand i10': 1.660,
#     'i20': 1.775,
#     'Aura/Xcent': 1.680,
#     'Creta': 1.790,
#     'Ioniq': 1.890,
#     'Exter': 1.710,
#     'Verna': 1.765,
#     'Tuscon': 1.850,
#     'Venue': 1.770,
#     'Kona': 1.800,
#     'New Verna': 1.765,
#     'Santro': 1.645,
#     'Elantra': 1.800,
#     'Alcazar': 1.790
# }
#     # Add other vehicle models and their dimensions here

# df2['total_length'] = df2.apply(lambda row: sum(vehicle_lengths[model] for model in df2.columns[3:] if model in vehicle_lengths), axis=1)

# # Calculate total width for each truck in df2
# df2['total_width'] = df2.apply(lambda row: sum(vehicle_widths[model] for model in df2.columns[3:] if model in vehicle_widths), axis=1)

# # Now df2 contains the additional columns total_length and total_width
# df2

In [1625]:
# # Assuming you have already loaded df1 and df2

# # Create a dictionary to map model names to their lengths and widths
# model_info = {
#    'Grand i10': {'Length(m)': 3.765, 'Width(m)': 1.660},
#     'i20': {'Length(m)': 3.995, 'Width(m)': 1.775},
#     'Aura/Xcent': {'Length(m)': 3.995, 'Width(m)': 1.680},
#     'Creta': {'Length(m)': 4.330, 'Width(m)': 1.790},
#     'Ioniq': {'Length(m)': 4.635, 'Width(m)': 1.890},
#     'Exter': {'Length(m)': 3.815, 'Width(m)': 1.710},
#     'Verna': {'Length(m)': 4.535, 'Width(m)': 1.765},
#     'Tuscon': {'Length(m)': 4.480, 'Width(m)': 1.850},
#     'Venue': {'Length(m)': 3.995, 'Width(m)': 1.770},
#     'Kona': {'Length(m)': 4.180, 'Width(m)': 1.800},
#     'New Verna': {'Length(m)': 4.535, 'Width(m)': 1.765},
#     'Santro': {'Length(m)': 3.610, 'Width(m)': 1.645},
#     'Elantra': {'Length(m)': 4.570, 'Width(m)': 1.800},
#     'Alcazar': {'Length(m)': 4.500, 'Width(m)': 1.790}
# }

# # Calculate total length and total width for each row in df2
# df2['total_length'] = df2.apply(lambda row: sum(model_info[model]['Length(m)'] * row[model] for model in model_info), axis=1)
# df2['total_width'] = df2.apply(lambda row: sum(model_info[model]['Width(m)'] * row[model] for model in model_info), axis=1)

# # Print the modified df2
# df2
